In [1]:
import numpy as np
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
import scanpy as sc
import scipy

In [2]:
sc.settings.verbosity = 3 

In [3]:
adata_raw = ad.read_h5ad("/Users/apple/Desktop/KB/data/LarryData/larry_raw_w_clone_id.h5ad")

In [4]:
len(adata_raw.obs["clone_id"].unique())

5864

In [5]:
adata_raw.obs['clone_id'].value_counts()

clone_id
1261    177
2370    165
5714    142
292     134
5209    130
       ... 
5594      2
866       2
4884      2
5101      2
2251      2
Name: count, Length: 5864, dtype: int64

In [6]:
value_counts = adata_raw.obs['clone_id'].value_counts()
frequency_dict = {}
for value, count in value_counts.items():
    if count in frequency_dict:
        frequency_dict[count].append(value)
    else:
        frequency_dict[count] = [value]



In [7]:
clone_for_remove = frequency_dict[2]+frequency_dict[3]+frequency_dict[4] 
adata_subset = adata_raw[~adata_raw.obs['clone_id'].isin(clone_for_remove)]

In [8]:
adata_subset.shape

(41201, 23420)

In [9]:
sc.pp.filter_cells(adata_subset, min_genes=200)
sc.pp.filter_genes(adata_subset, min_cells=3)

filtered out 92 cells that have less than 200 genes expressed


/opt/anaconda3/envs/scvi-env/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:139: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


filtered out 9031 genes that are detected in less than 3 cells


In [10]:
adata_subset.shape

(41109, 14389)

In [12]:
sc.pp.normalize_total(adata_subset, target_sum=1e4)

normalizing counts per cell
    finished (0:00:00)


In [14]:
sc.pp.log1p(adata_subset)

In [15]:
sc.pp.highly_variable_genes(adata_subset,n_top_genes=2000)

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:02)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


/opt/anaconda3/envs/scvi-env/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']


In [16]:
adata_subset.shape

(41109, 14389)

In [17]:
hvgene = (adata_subset.var.highly_variable[adata_subset.var.highly_variable==True]).index
print("number of the highly variable genes:", len(hvgene))


number of the highly variable genes: 2000


In [18]:
adata_hvgene = adata_subset[:,hvgene]

In [20]:
adata_hvgene.obs['clone_id'].value_counts()

clone_id
1261    177
2370    165
5714    141
292     134
5209    129
       ... 
434       5
3234      4
1468      4
4890      4
2829      4
Name: count, Length: 2817, dtype: int64

In [22]:
val_counts = adata_hvgene.obs['clone_id'].value_counts()
freq_dict = {}
for value, count in val_counts.items():
    if count in freq_dict:
        freq_dict[count].append(value)
    else:
        freq_dict[count] = [value]


In [23]:
clone_remove = freq_dict[4] 
adata_hvgene = adata_hvgene[~adata_hvgene.obs['clone_id'].isin(clone_remove)]
adata_hvgene.obs['clone_id'].value_counts()

clone_id
1261    177
2370    165
5714    141
292     134
5209    129
       ... 
513       5
5629      5
1014      5
3998      5
4329      5
Name: count, Length: 2813, dtype: int64

In [24]:
adata_hvgene.shape

(41093, 2000)

In [26]:
adata_hvgene.write("Larry_41093_2000_norm_log.h5ad")